In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import datetime
from tqdm import tqdm
tqdm.pandas()

In [2]:
df  = pd.read_csv('../Datasets/Binance_BTCUSDT_d.csv', header=1, usecols=['unix','open','high','low','close', 'Volume BTC'], nrows=1680)
df.dropna(inplace=True, how='any')

In [3]:
df['date']=pd.to_datetime(df['unix'],unit='ms')
df.drop(columns='unix', inplace=True)


In [4]:
df.set_index('date', inplace=True)

In [5]:
strat1 = ta.Strategy(name='Strategie 1',
                     description='MACD, RSI, EMA12, EMA26, Stochastic Oscillator',
                     ta=[
                       {'kind':'macd', 'fast':12 , 'slow':26},
                       {'kind':'ema', 'length':12},
                       {'kind':'ema', 'length':26},
                       {'kind':'stoch'},
                       {'kind':'rsi'},
                       
                     ])

In [6]:
df.ta.strategy(strat1)


In [7]:
df = df.iloc[33:]

In [8]:
df['diff'] = df['close'] - df['open']

/var/folders/51/frnt2kf50852hky9z_b7jx3m0000gn/T/ipykernel_79992/462976041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff'] = df['close'] - df['open']


In [9]:
def simplifier(val):
  if(val > 50):
    return 2
  elif(val<-50):
    return 0
  return 1
df['y'] = df['diff'].progress_apply(simplifier)

100%|██████████| 1647/1647 [00:00<00:00, 562679.70it/s]
/var/folders/51/frnt2kf50852hky9z_b7jx3m0000gn/T/ipykernel_79992/2545719035.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = df['diff'].progress_apply(simplifier)


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df = df[::-1]

In [12]:
X = df.drop(columns=['diff','y'])
y = df['y']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, train_size=0.9)

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
clf = GradientBoostingClassifier(criterion='friedmen_mse',learning_rate=0.75,loss='log_loss',
                                 max_depth=3,max_features='log2',min_samples_leaf=0.1,min_samples_split=0.1,
                                 n_estimators=100, subsample=0.5, random_state=1).fit(X_train,y_train)
clf.score(X_test,y_test)

ValueError: criterion='friedmen_mse' is not supported. Use criterion='friedman_mse' or 'squared_error' instead, as trees should use a squared error criterion in Gradient Boosting.

In [ ]:
#Criterion:friedmen_mse
#learning_rate:0.75
#loss:log_loss
#max_depth:3
#max_features:log2
#min_samples_leaf:0.1
#min_samples_split:0.1
#param_n_estimators:100
#parm_subsample:0.5
